In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [2]:
data = pd.read_csv("dynamic.csv")
data

,id,charttime,albumin,globulin,total_protein,aniongap,bicarbonate,bun,calcium,chloride,...,sodium_bg,lactate_bg,glucose_bg,d_dimer,fibrinogen,thrombin,inr,pt,ptt,urineoutput
0,28793466,4/12/29 3:35,NaN,NaN,NaN,16.0,21.0,12.0,8.8,106.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,10.9,27.8,NaN
1,25611175,11/29/40 3:50,NaN,NaN,NaN,15.0,27.0,28.0,8.5,101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,26115624,9/7/50 0:22,3.5,NaN,NaN,12.0,22.0,9.0,7.9,111.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.2,13.9,25.8,NaN
3,28164589,3/11/59 1:11,NaN,NaN,NaN,12.0,34.0,58.0,8.3,103.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.8,41.7,40.2,NaN
4,26115624,9/1/50 20:14,NaN,NaN,NaN,NaN,NaN,11.0,7.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93599,24995393,5/3/84 4:04,NaN,NaN,NaN,15.0,20.0,26.0,7.7,107.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.2,12.9,35.2,NaN
93600,23078014,8/10/42 5:04,NaN,NaN,NaN,8.0,27.0,19.0,8.9,105.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93601,24669863,2/18/88 3:00,NaN,NaN,NaN,15.0,27.0,16.0,9.3,101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.1,12.6,29.3,NaN
93602,27382401,12/20/41 6:27,NaN,NaN,NaN,22.0,17.0,27.0,6.4,103.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.5,16.4,60.1,NaN


In [3]:
data['charttime'] = pd.to_datetime(data['charttime'], errors='coerce')
data = data.sort_values(['id', 'charttime'])

In [4]:
missing_values_count = data.isnull().sum()
columns_with_less_missing = missing_values_count <= 0.5*len(data)

# 使用布尔索引选取符合条件的列
timeseries_data = data.loc[:, columns_with_less_missing]
timeseries_data

,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,...,mch,mchc,mcv,platelet,rbc,rdw,wbc,inr,pt,ptt
23353,20001305,1978-03-25 08:20:00,15.0,23.0,47.0,11.4,108.0,0.8,154.0,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52611,20001305,1978-03-25 13:45:00,13.0,25.0,48.0,10.8,107.0,0.9,149.0,140.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3,13.9,31.7
11891,20001305,1978-03-25 21:55:00,13.0,24.0,50.0,10.8,108.0,0.9,131.0,141.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3,14.1,33.1
66184,20001361,2043-05-04 17:24:00,14.0,22.0,28.0,6.3,107.0,2.5,161.0,137.0,...,30.7,34.1,90.0,216.0,3.76,13.1,20.8,1.8,19.4,33.5
78361,20001361,2043-05-04 21:07:00,15.0,20.0,32.0,6.5,108.0,2.5,124.0,137.0,...,30.3,34.0,89.0,227.0,4.03,13.2,17.8,1.6,17.6,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53583,29999498,1988-05-13 13:25:00,20.0,22.0,13.0,11.2,100.0,1.2,76.0,137.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4190,29999498,1988-05-14 03:26:00,15.0,21.0,13.0,10.1,103.0,1.2,102.0,135.0,...,30.9,32.4,95.0,231.0,2.96,16.8,90.4,1.2,12.7,29.8
50217,29999625,2057-11-07 13:33:00,17.0,21.0,14.0,8.5,104.0,1.2,109.0,142.0,...,28.2,32.9,86.0,256.0,5.82,14.1,15.1,1.2,13.2,25.4
29242,29999625,2057-11-08 01:36:00,15.0,24.0,21.0,8.6,110.0,1.6,122.0,149.0,...,28.5,33.1,86.0,256.0,5.54,14.6,16.7,NaN,NaN,NaN


In [5]:
timeseries_data = timeseries_data.groupby('id').apply(lambda group: group.ffill().bfill())
timeseries_data.reset_index(drop=True, inplace=True) 
# 将 'charttime' 设置为索引
#timeseries_data.set_index('charttime', inplace=True)
timeseries_data

,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,...,mch,mchc,mcv,platelet,rbc,rdw,wbc,inr,pt,ptt
0,20001305,1978-03-25 08:20:00,15.0,23.0,47.0,11.4,108.0,0.8,154.0,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3,13.9,31.7
1,20001305,1978-03-25 13:45:00,13.0,25.0,48.0,10.8,107.0,0.9,149.0,140.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3,13.9,31.7
2,20001305,1978-03-25 21:55:00,13.0,24.0,50.0,10.8,108.0,0.9,131.0,141.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3,14.1,33.1
3,20001361,2043-05-04 17:24:00,14.0,22.0,28.0,6.3,107.0,2.5,161.0,137.0,...,30.7,34.1,90.0,216.0,3.76,13.1,20.8,1.8,19.4,33.5
4,20001361,2043-05-04 21:07:00,15.0,20.0,32.0,6.5,108.0,2.5,124.0,137.0,...,30.3,34.0,89.0,227.0,4.03,13.2,17.8,1.6,17.6,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93599,29999498,1988-05-13 13:25:00,20.0,22.0,13.0,11.2,100.0,1.2,76.0,137.0,...,30.9,32.4,95.0,231.0,2.96,16.8,90.4,1.2,12.7,29.8
93600,29999498,1988-05-14 03:26:00,15.0,21.0,13.0,10.1,103.0,1.2,102.0,135.0,...,30.9,32.4,95.0,231.0,2.96,16.8,90.4,1.2,12.7,29.8
93601,29999625,2057-11-07 13:33:00,17.0,21.0,14.0,8.5,104.0,1.2,109.0,142.0,...,28.2,32.9,86.0,256.0,5.82,14.1,15.1,1.2,13.2,25.4
93602,29999625,2057-11-08 01:36:00,15.0,24.0,21.0,8.6,110.0,1.6,122.0,149.0,...,28.5,33.1,86.0,256.0,5.54,14.6,16.7,1.2,13.2,25.4


In [6]:
data1 = pd.read_csv("static.csv")
data1['hosp_admittime'] = pd.to_datetime(data1['hosp_admittime'], errors='coerce')
data1['hosp_dischtime'] = pd.to_datetime(data1['hosp_dischtime'], errors='coerce')
data1['icu_intime'] = pd.to_datetime(data1['icu_intime'], errors='coerce')
data1['icu_outtime'] = pd.to_datetime(data1['icu_outtime'], errors='coerce')
data1

,id,hosp_admittime,hosp_dischtime,icu_intime,icu_outtime,los_icu,icu_death,gender,race,admission_age,...,atrial_fibrillation,malignant_cancer,chf,ckd,cld,copd,diabetes,hypertension,ihd,stroke
0,24099382,2011-06-11 07:15:00,2011-06-18 21:45:00,2011-06-11 14:31:00,2011-06-13 18:05:00,2.15,0,M,OTHER,58.441631,...,0,0,0,0,0,1,1,1,0,0
1,27824879,2028-09-30 20:55:00,2028-10-10 15:54:00,2028-09-30 20:58:00,2028-10-05 18:39:00,4.90,0,F,WHITE,79.748598,...,0,0,0,0,0,1,0,1,1,0
2,27146619,2041-09-20 00:34:00,2041-09-22 12:05:00,2041-09-20 07:13:00,2041-09-21 18:10:00,1.46,0,F,WHITE,81.717397,...,0,0,1,1,0,0,0,0,1,0
3,22476441,2073-10-08 16:46:00,2073-10-16 13:05:00,2073-10-08 18:28:00,2073-10-10 13:26:00,1.79,0,F,BLACK/AFRICAN AMERICAN,63.769278,...,0,0,1,1,0,1,1,1,1,0
4,23169158,2027-03-12 01:19:00,2027-03-19 13:15:00,2027-03-12 02:43:00,2027-03-15 01:53:00,2.97,0,F,WHITE,84.191979,...,1,0,1,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20409,29968077,2059-09-12 06:20:00,2059-10-05 15:41:00,2059-09-12 07:45:00,2059-09-18 16:55:00,6.38,0,M,WHITE,73.695489,...,1,0,1,1,0,0,0,1,1,0
20410,29975784,2036-08-30 15:18:00,2036-09-02 16:15:00,2036-08-30 16:48:00,2036-09-01 10:41:00,1.75,0,M,WHITE,86.663745,...,0,0,1,1,0,0,0,1,0,0
20411,29987115,2048-02-16 17:42:00,2048-02-20 19:00:00,2048-02-16 17:43:00,2048-02-20 21:00:00,4.14,1,F,UNKNOWN,43.127963,...,0,0,1,0,0,0,0,0,0,0
20412,29990494,2071-05-10 12:05:00,2071-06-16 13:45:00,2071-05-17 22:16:00,2071-05-28 20:41:00,10.93,0,M,WHITE - BRAZILIAN,57.354082,...,0,0,0,1,0,0,0,1,0,0


In [7]:
merged_data = pd.merge(timeseries_data, data1, on='id', how='left')
merged_data

,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,...,atrial_fibrillation,malignant_cancer,chf,ckd,cld,copd,diabetes,hypertension,ihd,stroke
0,20001305,1978-03-25 08:20:00,15.0,23.0,47.0,11.4,108.0,0.8,154.0,142.0,...,0,0,0,0,0,1,0,1,0,0
1,20001305,1978-03-25 13:45:00,13.0,25.0,48.0,10.8,107.0,0.9,149.0,140.0,...,0,0,0,0,0,1,0,1,0,0
2,20001305,1978-03-25 21:55:00,13.0,24.0,50.0,10.8,108.0,0.9,131.0,141.0,...,0,0,0,0,0,1,0,1,0,0
3,20001361,2043-05-04 17:24:00,14.0,22.0,28.0,6.3,107.0,2.5,161.0,137.0,...,0,0,0,0,0,0,0,0,0,0
4,20001361,2043-05-04 21:07:00,15.0,20.0,32.0,6.5,108.0,2.5,124.0,137.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93599,29999498,1988-05-13 13:25:00,20.0,22.0,13.0,11.2,100.0,1.2,76.0,137.0,...,0,0,0,1,0,0,0,1,0,0
93600,29999498,1988-05-14 03:26:00,15.0,21.0,13.0,10.1,103.0,1.2,102.0,135.0,...,0,0,0,1,0,0,0,1,0,0
93601,29999625,2057-11-07 13:33:00,17.0,21.0,14.0,8.5,104.0,1.2,109.0,142.0,...,1,0,1,0,0,0,0,1,0,0
93602,29999625,2057-11-08 01:36:00,15.0,24.0,21.0,8.6,110.0,1.6,122.0,149.0,...,1,0,1,0,0,0,0,1,0,0


In [8]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Let's assume 'df' is your DataFrame.

# Separate the dataframe into numeric and non-numeric data
numeric_df = merged_data.select_dtypes(include='number')
non_numeric_df = merged_data.select_dtypes(exclude='number')

# Apply the IterativeImputer to numeric data
imp = IterativeImputer(max_iter=10, random_state=0)
numeric_df_imputed = imp.fit_transform(numeric_df)

# Convert the imputed array back to a DataFrame
numeric_df_imputed = pd.DataFrame(numeric_df_imputed, columns=numeric_df.columns, index=numeric_df.index)

# Combine the numeric and non-numeric data back into a single DataFrame
df_imputed = pd.concat([numeric_df_imputed, non_numeric_df], axis=1)

# Ensure the original order of columns is preserved
df_imputed = df_imputed[merged_data.columns]
df_imputed = df_imputed.reset_index(drop=True)
df_imputed

,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,...,atrial_fibrillation,malignant_cancer,chf,ckd,cld,copd,diabetes,hypertension,ihd,stroke
0,20001305.0,1978-03-25 08:20:00,15.0,23.0,47.0,11.4,108.0,0.8,154.0,142.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,20001305.0,1978-03-25 13:45:00,13.0,25.0,48.0,10.8,107.0,0.9,149.0,140.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,20001305.0,1978-03-25 21:55:00,13.0,24.0,50.0,10.8,108.0,0.9,131.0,141.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,20001361.0,2043-05-04 17:24:00,14.0,22.0,28.0,6.3,107.0,2.5,161.0,137.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20001361.0,2043-05-04 21:07:00,15.0,20.0,32.0,6.5,108.0,2.5,124.0,137.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93599,29999498.0,1988-05-13 13:25:00,20.0,22.0,13.0,11.2,100.0,1.2,76.0,137.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
93600,29999498.0,1988-05-14 03:26:00,15.0,21.0,13.0,10.1,103.0,1.2,102.0,135.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
93601,29999625.0,2057-11-07 13:33:00,17.0,21.0,14.0,8.5,104.0,1.2,109.0,142.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
93602,29999625.0,2057-11-08 01:36:00,15.0,24.0,21.0,8.6,110.0,1.6,122.0,149.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [9]:

are_there_still_missing_values = df_imputed.isnull().any().any()

# 打印检查结果
print(are_there_still_missing_values)


False


In [10]:
#df_imputed.drop('hosp_admittime', axis=1, inplace=True)
#df_imputed.drop('hosp_dischtime', axis=1, inplace=True)

df_imputed['charttime_month'] = df_imputed['charttime'].dt.month 
df_imputed['charttime_day'] = df_imputed['charttime'].dt.day 
df_imputed['charttime_hour'] = df_imputed['charttime'].dt.hour 
#df_imputed.drop('charttime', axis=1, inplace=True)

df_imputed['icu_intime_month'] = df_imputed['icu_intime'].dt.month
df_imputed['icu_intime_day'] = df_imputed['icu_intime'].dt.day
df_imputed['icu_intime_hour'] = df_imputed['icu_intime'].dt.hour
#df_imputed.drop('icu_intime', axis=1, inplace=True)

df_imputed['icu_outtime_month'] = df_imputed['icu_outtime'].dt.month 
df_imputed['icu_outtime_day'] = df_imputed['icu_outtime'].dt.day 
df_imputed['icu_outtime_hour'] = df_imputed['icu_outtime'].dt.hour 
#df_imputed.drop('icu_outtime', axis=1, inplace=True)

df_imputed

,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,...,stroke,charttime_month,charttime_day,charttime_hour,icu_intime_month,icu_intime_day,icu_intime_hour,icu_outtime_month,icu_outtime_day,icu_outtime_hour
0,20001305.0,1978-03-25 08:20:00,15.0,23.0,47.0,11.4,108.0,0.8,154.0,142.0,...,0.0,3,25,8,3,25,2,3,27,21
1,20001305.0,1978-03-25 13:45:00,13.0,25.0,48.0,10.8,107.0,0.9,149.0,140.0,...,0.0,3,25,13,3,25,2,3,27,21
2,20001305.0,1978-03-25 21:55:00,13.0,24.0,50.0,10.8,108.0,0.9,131.0,141.0,...,0.0,3,25,21,3,25,2,3,27,21
3,20001361.0,2043-05-04 17:24:00,14.0,22.0,28.0,6.3,107.0,2.5,161.0,137.0,...,0.0,5,4,17,5,4,16,5,10,17
4,20001361.0,2043-05-04 21:07:00,15.0,20.0,32.0,6.5,108.0,2.5,124.0,137.0,...,0.0,5,4,21,5,4,16,5,10,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93599,29999498.0,1988-05-13 13:25:00,20.0,22.0,13.0,11.2,100.0,1.2,76.0,137.0,...,0.0,5,13,13,5,13,12,5,14,16
93600,29999498.0,1988-05-14 03:26:00,15.0,21.0,13.0,10.1,103.0,1.2,102.0,135.0,...,0.0,5,14,3,5,13,12,5,14,16
93601,29999625.0,2057-11-07 13:33:00,17.0,21.0,14.0,8.5,104.0,1.2,109.0,142.0,...,0.0,11,7,13,11,7,11,11,29,15
93602,29999625.0,2057-11-08 01:36:00,15.0,24.0,21.0,8.6,110.0,1.6,122.0,149.0,...,0.0,11,8,1,11,7,11,11,29,15


In [11]:

# 假设 df 是你的DataFrame
# 识别非数值型列
categorical_columns = df_imputed.select_dtypes(include=['object']).columns

# 应用独热编码
df_imputed = pd.get_dummies(df_imputed, columns=categorical_columns)

# df_encoded 现在包含原始数值列以及独热编码的非数值型列
df_imputed

,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,...,admission_type_URGENT,first_careunit_Cardiac Vascular Intensive Care Unit (CVICU),first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU)
0,20001305.0,1978-03-25 08:20:00,15.0,23.0,47.0,11.4,108.0,0.8,154.0,142.0,...,0,0,0,0,1,0,0,0,0,0
1,20001305.0,1978-03-25 13:45:00,13.0,25.0,48.0,10.8,107.0,0.9,149.0,140.0,...,0,0,0,0,1,0,0,0,0,0
2,20001305.0,1978-03-25 21:55:00,13.0,24.0,50.0,10.8,108.0,0.9,131.0,141.0,...,0,0,0,0,1,0,0,0,0,0
3,20001361.0,2043-05-04 17:24:00,14.0,22.0,28.0,6.3,107.0,2.5,161.0,137.0,...,0,0,0,0,1,0,0,0,0,0
4,20001361.0,2043-05-04 21:07:00,15.0,20.0,32.0,6.5,108.0,2.5,124.0,137.0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93599,29999498.0,1988-05-13 13:25:00,20.0,22.0,13.0,11.2,100.0,1.2,76.0,137.0,...,0,0,0,0,1,0,0,0,0,0
93600,29999498.0,1988-05-14 03:26:00,15.0,21.0,13.0,10.1,103.0,1.2,102.0,135.0,...,0,0,0,0,1,0,0,0,0,0
93601,29999625.0,2057-11-07 13:33:00,17.0,21.0,14.0,8.5,104.0,1.2,109.0,142.0,...,0,0,0,0,0,0,0,1,0,0
93602,29999625.0,2057-11-08 01:36:00,15.0,24.0,21.0,8.6,110.0,1.6,122.0,149.0,...,0,0,0,0,0,0,0,1,0,0


In [36]:
icu_death_0 = df_imputed[df_imputed['icu_death'] == 0]
icu_death_1 = df_imputed[df_imputed['icu_death'] == 1]
icu_death_0 = icu_death_0.reset_index(drop=True)
icu_death_1 = icu_death_1.reset_index(drop=True)

In [12]:
df_imputed.to_csv('df_imputed_nodrop.csv')